In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Markov Decision Process Recap
Before we talk about Q-Learning, let's first recap what we know about Markov Decision Processes(MDP's). An MDP can be represented with a 5-tuple $(S, A, R, P, \gamma)$ where
* $S$ is the state space
* $A$ is the action space
* $R$ is the reward function $R: S \times A \rightarrow \mathbb{R}$
* $P$ is the transition probability function. This tells you the probability of ending up in state $'s$ when you take an action $a$ from state $s$. $P: S \times A \times S \rightarrow [0, 1]$
* $\gamma \in [0, 1]$ is the discount factor which tells you how much to value future rewards

## Goal
The goal in MDP problems is to find a **policy** which we denote $\pi$ to maximize our long term reward. Suppose we have some arbitrary agent going through the MDP that sees states $s_1, s_2, ...$ and takes actions $a_1, a_2, ...$
in each time step. One way to think about this agents total reward is to simply sum up the rewards observed at
each time step:
$$
R_{total} = R(s_1, a_1) + R(s_2, a_2) + ... + R(s_{T}, a_{T}) = \sum_{t=1}^{T} R(s_t, a_t)
$$
For notational convenience, going forward we'll sometimes instead use $R_t$ to denote the reward at timestep $t$. Or $R_t = R(s_t, a_t)$, or the reward acrued at timestep $t$
This finite summation makes sense when our MDP is finite (aka there is some terminal state). This is the case in games like checkers, chess, go. The number of time steps $T$ is a random variable can vary from episode to episode.

You might wonder, how about MDP's that have an infinite time window. For example, the cartpole environment that we saw in the previous notebook can be considered an infinite time horizon MDP - you could imagine that with the right actions at each time step, you can balance the pole on the cart forever.

So for the infinite MDP case, we can instead consider the **discounted long term reward**:
$$
R_{total} = R(s_1, a_1) + \gamma R(s_2, a_2) + ... + \gamma^{T-1} R(s_{T}, a_{T}) = \sum_{t=1}^T \gamma^{t-1} R(s_t, a_t) 
$$
Why might we want to use the discoiunted long term reward instead of the raw total reward?
* The discounting factor ensures that this summation converges to a finite value
* The discounting allows us to formalize the notion that future states/rewards are not as important as the current state/current reward.

Consider the extreme case when $\gamma = 0$, this says that the most important thing to do is to pick the best action for the current state, regardless of what happens in the future.

When $\gamma = 1$, then we are saying that every time step matters equally. Note that for $\gamma = 1$, this is identical to the long term reward defined above, so going forward we will only talk about trying to maximize the discounted long term reward.

## Value Functions
Most reinforcement learning algorithms, either directly or indirectly, try to model a **value function**, which tells you how good some given state is or how good a given state, action pair is. In most reinforcement learning references you'll see these written as:
$$
V: S \rightarrow \mathbb{R} \\
Q: S \times A \rightarrow{R}
$$
$V(s)$ tells you how "good" it is to be in a given state $s$, and $Q(s, a)$ tells you how good it is to take action $a$ when you're in state $s$.

Suppose you have somehow have a perfect $Q$ function. You can use this $Q$ function to inform the actions you take at each state. IE: you start at state $s_1$, then from there, pick the action $a_1$ that maximizes $Q(s, a)$. From there, you end up in state $s_2$, and then you pick the action $a_2$ that maximizes $Q(s_2, a_2)$ and so on. So, this suggests that if we have a $V$ or $Q$ function, that gives us a policy to follow. Most reinforcement learning algorithms boil down to accurately modeling a $Q$ function.

At this point you should be a bit cautious and think, well, how good a state/action pair is should also depend on what actions that you/the agent takes in the future, right? Consider the following simple example:

### The $n \times 2$ Cliff Environment:
Suppose you're walking along some horizontal cliff that goes east/west. You start off at the west most location and your goal is to make it to east most spot of this cliff. There is a steep rock formation to the south of this horizontal strip which makes it impossible to go south.  North of the cliff is a 1000 foot drop to the ocean. So we can represent this with a 2 x n grid where you start off at location (0, 0) and you want to get to $(0, n-1)$. In this environment, our episode ends whenever the agent moves North and falls off the cliff, or reaches the target location $(0, n-1)$.

Here, our action space is: $$A = \{N, E, W\}$$ Note that we omit $S$ as an action because once we move North once,
the episode will end because you're now dead so you can never actually walk south.

Our states can be represented with tuples that give our current location in the grid: $$S = \{ (x, y): 0 \leq x, y \leq n-1\, \text{ and } x, y \in \mathbb{Z} \}$$

We can cook up a reward function for this MDP. Any action that where you step off the cliff to your death gives you a reward of $-1$, any that keeps you alive gets you a reward of $0$, any action that gets you to the target location $(0, n-1)$ gets you a reward of $1$. So formally we can say:
$$
R((0, y) , N) = -1 \\  \\
R((0, y), E) = 0, \text{ for } 0 \leq y < n-3 \\
R((0, y), E) = 1, \text{ for } y = n-2 \\
R((0, y), W) = 0, \text{ for } 1 \leq y < n-1 \\
$$
TODO: make this latex look nice

Going back to value functions, you can imagine, well being in state $(0, n-2)$ should be really good because you're just one step away from the target location $(0, n-1)$. But it is not so great if you have some dumb policy that always moves North in that state (I guess in that case, this policy can never get your agent to the target location). So really these $V$ and $Q$ functions should be parameterized by the **policy** that you/your agent is following.


## Sample code for the Cliff environment
A very simple example of an MDP is a maze game, where you(the agent) is plopped down in some rectangular grid and your goal is to eventually find some hidden cheese in the grid.

In [63]:
class GridEnv(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = np.zeros((width, height))
    def step(self, action):
        '''
        Things that subclass this should return a 3-tuple (new_state, reward, done)
        new_state: the new state
        reward: how much reward you get for taking that action
        done: boolean that tells you if the environment is done
        '''
        raise("Not Implemented!")
        
    def available_actions(self, state):
        raise("Not Implemented!")
        
class Cliff(GridEnv):
    def __init__(self, width):
        super(Cliff, self).__init__(width, 2)
        
        self.target_location = (0, width-1)
        self.reset()
        
    def get_reward(self, state, action):
        if action is 'N':
            return -1
        
    def step(self, action):
        assert action in ('N', 'E', 'W'), 'Action given: {} is not one of the acceptable actions!'.format(action)
        assert action in self.available_actions(self.state), 'Action {} is not allowed in state {}'.format(action, self.state)        
        
        reward = self._reward(self.state, action)
        done = None
        self.grid[self.state] = 0
        if action is 'N':
            self.state = (self.state[0], self.state[1] + 1)
            done = True
        elif action is 'W':
            self.state = (self.state[0] - 1, self.state[1])
            done = False
        elif action is 'E':
            print('going east!')
            self.state = (self.state[0] + 1, self.state[1])
            if self.state == self.target_location:
                done = True
            else:
                done = False
        else:
            raise Exception("Should never be in this state!")
    
        # update grid
        self.grid[self.state] = 1
        return self.state, reward, done
    
    def available_actions(self, state):
        x, y = state
        actions = []
        if y == 0:
            actions.append('N')
            if x > 0:
                actions.append('W')
            if x < self.width - 1:
                actions.append('E')
        if y == 1:
            # well you'll never really be here because the episode will have terminated
            pass
            
        return actions
    
    def _reward(self, state, action):
        '''
        Note that in general, the agent has no access to the true reward except by
        taking actions in the environment.
        
        '''
        # assume this is an allowed state/action
        if action is 'N':
            return -1
        if action is 'W':
            return 0
        if action is 'E' and state is (0, self.width-2):
            return 1
        if action is 'E':
            return 0
        return 0
        
    def reset(self):
        self.state = (0, 0)
        self.grid[self.state] = 1

    def state(self):
        return self.grid
    
    def show(self):
        plt.imshow(self.grid, origin='lower')
        plt.show()

In [68]:
def test():
    cliff = Cliff(2)
    state, reward, done = cliff.step('N')
    print('New state: {}, reward: {}, done: {}'.format(state, reward, done))
    cliff.reset()
    print(cliff.state)
    state, reward, done = cliff.step('E')
    print('New state: {}, reward: {}, done: {}'.format(state, reward, done))
    cliff.reset()
test()

New state: (0, 1), reward: -1, done: True
(0, 0)
going east!
New state: (1, 0), reward: 0, done: False


## Policies

When you're in a given state, you can think of a policy as defining a probability distribution over your set of actions. In the cliff example above, the optimal policy is to always move east. So the best policy assigns a probability of going east at each valid state to $1$ and the probability of every other action in each valid state as 0. Formally, we denote a policy $\pi: S \times A \rightarrow [0, 1]$ as a function from the set of state, action pairs to a value between $0$ and $1$ and for any $s \in S, \sum_{a \in A} \pi(s, a) = 1$.

## Defining the Value Functions in terms of a policy
Now we can define the $V$ and the $Q$ functions in terms of a policy $\pi$. To make the dependence on $\pi$ explicit, we will place the $\pi$ in the superscript. $V^{\pi}, Q^{\pi}$. In some texts you'll see the $\pi$ in the subscript $V_\pi, Q_\pi$. I think the superscript looks nicer though so I'll stick with the superscript.

Then the value functions are defined as the _expected_ long term discounted reward under policy $\pi$. That is, how much reward can you expect to accrue if you you take action $a$ with probability given by your policy $\pi(s)$ at each state $s$.
$$
V^{\pi}(s) = \mathbb{E}_{\pi} [ \sum_{t=1}^{\infty} \gamma^{t-1} R_t]
$$

Similarly, we can define the $Q^{\pi}$ function as the value of taking action $a$ in state $s$
following policy $\pi$. At state $s$ you take action $a$, accrue some immediate reward, and then henceforth, you follow policy $\pi$.
$$
Q^{\pi} (s, a) = R(s, a) + \gamma \mathbb{E}_{s' \sim P(s, a, s'), \pi} [\sum_{t=1}^\infty \gamma^{t-1} R_t]
$$
To unpack the notation a bit, after taking action $a$ in state $s$, we'll end up in some new state $s'$ which is drawn from our transition probability distribution $P$ (recall that the way that we transition states is one of the components of the 5-tuple that defines an MDP).
Abusing notation a bit, lets we denote $P(s, a)$ as the probability distribution over states we might end up in after taking action $a$ in state $s$. Then we can also rewrite the $Q^{\pi}$ function in terms of $V^{\pi}$ in the following way:

$$
Q^{\pi}(s, a) = R(s, a) + \gamma \mathbb{E}[V^{\pi}(s')], \text{ where the expectation is over $s'$ is drawn from $P(s, a)$}
$$

It is a good exercise to rewrite the $V^{\pi}$ function in a recursive way as well: the value of state $s$ under a policy $\pi$ is given by the immedidate reward you'd get $R(s, a)$, where $a$ is drawn from the distribution over actions defined by the policy $\pi(s)$ and then discounting the future reward from the new state $s'$ you end up in:

$$
V^{\pi}(s) = R(s, a) + \gamma \mathbb{E}[V^{\pi}(s')], \text{ where the expectation is over $s'$ drawn from $P(s, a)$}
$$

This recursive definition of $V^{\pi}$ and $Q^{\pi}$ will be key in developing an algorithm to simultaneously learn these functions and policies given these functions.
I'm playing a bit loose with notation but hopefully the big picture is clear.